# Sync tags to existing scenes

In [ ]:
import pandas as pd
import dotenv
import os
from libraries.client_stashapp import get_stashapp_client
from libraries.StashDbClient import StashDbClient

dotenv.load_dotenv()

stash = get_stashapp_client()

stashbox_client = StashDbClient(
    os.getenv("STASHDB_ENDPOINT"),
    os.getenv("STASHDB_API_KEY"),
)

In [ ]:
def get_scenes_with_stashdb_id():
    scenes = stash.find_scenes(
        {
            "stash_id_endpoint": {
                "endpoint": "https://stashdb.org/graphql",
                "modifier": "NOT_NULL",
            },
            "tags": {
                "value": [],
                "modifier": "IS_NULL"
            }
        },
        filter = {
            "per_page": 1000,
            "page": 2
        }
    )
    return scenes

scenes_with_stashdb_id = get_scenes_with_stashdb_id()
df_scenes = pd.DataFrame(scenes_with_stashdb_id)


In [ ]:
# Step 2: Extract StashDB IDs and existing tags
df_scenes['stashdb_id'] = df_scenes['stash_ids'].apply(
    lambda x: next((stash_id['stash_id'] for stash_id in x if stash_id['endpoint'] == 'https://stashdb.org/graphql'), None)
)
df_scenes['existing_tags'] = df_scenes['tags'].apply(lambda x: sorted([tag['name'] for tag in x]))

# Step 3: Fetch StashDB data for each scene
def get_stashdb_data(stashdb_id):
    return stashbox_client.query_scenes(stashdb_id)

df_scenes['stashdb_data'] = df_scenes['stashdb_id'].apply(get_stashdb_data)

# Step 4: Extract tags from StashDB data
def extract_tags(stashdb_data):
    if stashdb_data and 'data' in stashdb_data and 'findScene' in stashdb_data['data']:
        return sorted([tag['name'] for tag in stashdb_data['data']['findScene']['tags']])
    return []

df_scenes['stashdb_tags'] = df_scenes['stashdb_data'].apply(extract_tags)

# Step 5: Merge existing tags with StashDB tags
df_scenes['merged_tags'] = df_scenes.apply(lambda row: sorted(list(set(row['existing_tags'] + row['stashdb_tags']))), axis=1)

# Step 6: Compare existing tags with merged tags
df_scenes['tags_identical'] = df_scenes.apply(lambda row: set(row['existing_tags']) == set(row['merged_tags']), axis=1)

# Step 7: Find corresponding tags in local Stash for merged tags
def find_stash_tags(tag_names):
    return sorted([stash.find_tag(tag_name) for tag_name in tag_names], key=lambda x: x['name'] if x else '')

df_scenes['stash_tags'] = df_scenes['merged_tags'].apply(find_stash_tags)

# Step 8: Select relevant columns
df_selected = df_scenes[['id', 'title', 'details', 'date', 'existing_tags', 'stashdb_tags', 'merged_tags', 'tags_identical', 'stash_tags']]


: 

In [ ]:
# Update tags for scenes where tags are not identical
for index, row in df_selected.iterrows():
    if not row['tags_identical'] and row['stash_tags']:
        stash.update_scene({
            "id": row['id'],
            "tag_ids": [tag['id'] for tag in row['stash_tags'] if tag]
        })

In [ ]:
# Read a scene from Stash by ID
scene_id = 6775  # Replace with the actual scene ID
scene = stash.find_scene(scene_id)
stashdb_id = next((stash_id['stash_id'] for stash_id in scene['stash_ids'] if stash_id['endpoint'] == 'https://stashdb.org/graphql'), None)

# Print scene details
print(f"Scene Title: {scene['title']}")
print(f"Scene Date: {scene['date']}")
print(f"Scene Details: {scene['details']}")
print(f"StashDB ID: {stashdb_id}")



In [ ]:
scene_existing_tags = scene['tags']
scene_existing_tags

In [ ]:
stashdb_scene = stashbox_client.query_scenes(stashdb_id)
stashdb_scene


In [ ]:
stash_tags = []
for tag in stashdb_scene['data']['findScene']['tags']:
    stash_tag = stash.find_tag(tag["name"])
    stash_tags.append({ "id": stash_tag["id"], "name": stash_tag["name"], "description": stash_tag["description"] })

stash_tags

In [ ]:
stash.update_scene({
    "id": scene["id"],
    "tag_ids": [tag["id"] for tag in stash_tags]
})